<a href="https://colab.research.google.com/github/kacper40/MEDICA-/blob/main/ZPI_33984.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Formularz dodawania pacjenta i wyświetlanie danych

In [ ]:
# ⬅️ NAJPIERW: uruchom to w Colab
import sqlite3
from google.colab import output
from IPython.display import HTML, display, Javascript

# 👉 Inicjalizacja bazy i tabeli
conn = sqlite3.connect("pacjenci.db")
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS pacjenci (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  imie TEXT,
  wiek INTEGER,
  plec TEXT,
  notatki TEXT
)
''')
conn.commit()

# 👉 Funkcja do dodawania pacjenta
def zapisz_pacjenta(dane):
  cursor.execute('''
    INSERT INTO pacjenci (imie, wiek, plec, notatki) VALUES (?, ?, ?, ?)
  ''', (dane['name'], dane['age'], dane['gender'], dane['notes']))
  conn.commit()
  print("✅ Pacjent zapisany do bazy.")

# 👉 Obsługa komunikacji JS → Python
def odbierz_dane(z_json):
  zapisz_pacjenta(z_json)

output.register_callback('zapiszPacjenta', odbierz_dane)

# 👉 Formularz HTML + JavaScript
form_html = """
<div class="form-container" style="font-family:Arial; background:white; padding:20px; border-radius:10px; max-width:500px; box-shadow:0 0 10px rgba(0,0,0,0.1);">
  <h2 style="color:black;">Dodaj pacjenta</h2>
  <form onsubmit="wyslij(event)" style="color:black;">
    <label>Imię i nazwisko:</label>
    <input type="text" id="name" required style="width:90%;padding:10px;margin-bottom:10px;"><br>
    <label>Wiek:</label>
    <input type="number" id="age" required style="width:90%;padding:10px;margin-bottom:10px;"><br>
    <label>Płeć:</label>
    <select id="gender" style="width:95%;padding:10px;margin-bottom:10px;">
      <option value="male">Mężczyzna</option>
      <option value="female">Kobieta</option>
    </select><br>
    <label>Notatki:</label>
    <input type="text" id="notes" style="width:90%;padding:10px;margin-bottom:10px;"><br>
    <button type="submit" style="padding:10px 15px; background:#4CAF50; color:white; border:none; border-radius:5px;">Zapisz</button>
  </form>
  <div id="output"></div>
</div>

<script>
  function wyslij(e) {
    e.preventDefault();
    const dane = {
      name: document.getElementById("name").value,
      age: document.getElementById("age").value,
      gender: document.getElementById("gender").value,
      notes: document.getElementById("notes").value
    };
    google.colab.kernel.invokeFunction('zapiszPacjenta', [dane], {});
    document.getElementById("output").innerHTML = "<p style='color:green;'>✅ Zapisano pacjenta!</p>";
    e.target.reset();
  }
</script>
"""

display(HTML(form_html))


✅ Pacjent zapisany do bazy.


In [ ]:
from IPython.display import HTML

def pokaz_pacjentow_stylowo():
  # Pobranie danych z bazy
  cursor.execute("SELECT * FROM pacjenci")
  rows = cursor.fetchall()

  # Styl + tabela HTML
  html = """
  <style>
    .tabela-container {
      font-family: Arial;
      background: white;
      padding: 20px;
      margin: 20px auto;
      border-radius: 10px;
      max-width: 800px;
      box-shadow: 0 0 10px rgba(0,0,0,0.1);
    }
    table {
      width: 100%;
      border-collapse: collapse;
      margin-top: 10px;
    }
    th, td {
      border: 1px solid #ddd;
      padding: 12px;
      text-align: left;
      color: black;
    }
    th {
      background-color: #f2f2f2;
    }
    h2 {
      margin-top: 0;
    }
  </style>

  <div class="tabela-container">
    <h2 style="color:black;">Lista pacjentów</h2>
    <table>
      <tr>
        <th>ID</th>
        <th>Imię i nazwisko</th>
        <th>Wiek</th>
        <th>Płeć</th>
        <th>Notatki</th>
      </tr>
  """

  for row in rows:
    html += f"""
      <tr>
        <td>{row[0]}</td>
        <td>{row[1]}</td>
        <td>{row[2]}</td>
        <td>{row[3]}</td>
        <td>{row[4]}</td>
      </tr>
    """

  html += """
    </table>
  </div>
  """

  display(HTML(html))

# 👉 Uruchom wywołanie poniżej, aby wyświetlić pacjentów:
pokaz_pacjentow_stylowo()


ID,Imię i nazwisko,Wiek,Płeć,Notatki
1,Jolanta,51,female,
2,Tomasz,47,male,zdrów jak ryba
3,Anna Jakubiak,34,female,


# Formularz logowania

Stworzenie bazy danych z danymi do logowania

In [ ]:
# Nowe połączenie do bazy użytkowników
conn_users = sqlite3.connect("uzytkownicy.db")
cursor_users = conn_users.cursor()

# Tworzymy tabelę użytkowników w osobnej bazie
cursor_users.execute('''
CREATE TABLE IF NOT EXISTS uzytkownicy (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  login TEXT UNIQUE,
  haslo TEXT
)
''')

# Dodajemy przykładowe konta, jeśli nie istnieją
przykladowi = [
    ("admin", "admin123"),
    ("janek", "haslojanek"),
    ("anna", "annasecret")
]

for login, haslo in przykladowi:
  cursor_users.execute("INSERT OR IGNORE INTO uzytkownicy (login, haslo) VALUES (?, ?)", (login, haslo))

conn_users.commit()
print("✅ Nowa baza użytkowników została utworzona.")


✅ Nowa baza użytkowników została utworzona.


Formularz logowania

In [ ]:
from IPython.display import HTML

login_html = """
<div style="font-family: Arial; background: white; padding: 20px; border-radius: 10px; max-width: 400px; margin: 20px auto; box-shadow: 0 0 10px rgba(0,0,0,0.1);">
  <h2 style="color:black;">Logowanie</h2>
  <form onsubmit="zaloguj(event)">
    <label style="color:black;">Login:</label>
    <input type="text" id="login" required style="width:90%; padding:10px; margin-bottom:10px;"><br>
    <label style="color:black;">Hasło:</label>
    <input type="password" id="haslo" required style="width:90%; padding:10px; margin-bottom:10px;"><br>
    <button type="submit" style="padding:10px 15px; background:#4CAF50; color:white; border:none; border-radius:5px;">Zaloguj</button>
  </form>
  <div id="wynik-logowania" style="margin-top:10px;"></div>
</div>

<script>
function zaloguj(e) {
  e.preventDefault();
  const dane = {
    login: document.getElementById("login").value,
    haslo: document.getElementById("haslo").value
  };
  google.colab.kernel.invokeFunction('sprawdzLogowanie', [dane], {});
}
</script>
"""

display(HTML(login_html))

# Musi być wcześniej: conn_users = sqlite3.connect("uzytkownicy.db")
def sprawdz_logowanie_z_osobnej_bazy(dane):
  login = dane['login']
  haslo = dane['haslo']

  cursor_users.execute("SELECT * FROM uzytkownicy WHERE login=? AND haslo=?", (login, haslo))
  user = cursor_users.fetchone()

  if user:
    print(f"✅ Zalogowano użytkownika: {login}")
    output.eval_js("document.getElementById('wynik-logowania').innerHTML = '<p style=color:green>✅ Zalogowano pomyślnie!</p>'")
  else:
    print(f"❌ Nieprawidłowe dane logowania dla: {login}")
    output.eval_js("document.getElementById('wynik-logowania').innerHTML = '<p style=color:red>❌ Błędny login lub hasło!</p>'")

output.register_callback('sprawdzLogowanie', sprawdz_logowanie_z_osobnej_bazy)


✅ Zalogowano użytkownika: admin
❌ Nieprawidłowe dane logowania dla: jan11


#Schemat bazy danych

In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM pacjenci", conn)
df

,id,imie,wiek,plec,notatki
0,1,Jolanta,51,female,
1,2,Tomasz,47,male,zdrów jak ryba
2,3,Anna Jakubiak,34,female,
